In [284]:
# copyrights (c) 2024 - thyung
# GPLv3

from ipywidgets import *
import time
import threading
import math
import random

SCREEN_X = 11
SCREEN_Y = 11
GEN_GATES_AFTER_CYCLE = 4

def apply_gate_on_state(gate, state):
    if gate == 'X':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '+'
        elif state == '-':
            result = '-'
        elif state == '>':
            result = '<'
        elif state == '<':
            result = '>'
    elif gate == 'Y':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == '>':
            result = '>'
        elif state == '<':
            result = '<'
    elif gate == 'Z':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == '>':
            result = '<'
        elif state == '<':
            result = '>'
    elif gate == 'S':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = '>'
        elif state == '-':
            result = '<'
        elif state == '>':
            result = '-'
        elif state == '<':
            result = '+'
    elif gate == 'H':
        if state == '0':
            result = '+'
        elif state == '1':
            result = '-'
        elif state == '+':
            result = '0'
        elif state == '-':
            result = '1'
        elif state == '>':
            result = '<'
        elif state == '<':
            result = '>'

    return result

class QubitBoard:
    def __init__(self):
        self.states = ['0', '1', '+', '-', '<', '>']
        self.gates = ['X', 'Y', 'Z', 'S', 'H']
        self.exit = False
        self.build_ui()

    def build_ui(self):
        self.sld_frame_rate = FloatSlider(value=1.0, min=0.5, max=4, step=0.1, description='Frame rate:')
        self.sld_gates_rate = IntSlider(value=4, min=1, max=10, description='Gates rate:')
        self.tiles = [[Button(description=' ', layout=Layout(width='30px', height='30px'), disabled=True) for i in range(SCREEN_X)] 
                      for j in range(SCREEN_Y)]
        self.btn_stop = Button(description='Stop')
        self.btn_stop.on_click(lambda b: self.on_exit())
        boxes = [Box(i) for i in self.tiles]
        board = VBox(boxes)
        screen = VBox([self.sld_frame_rate, self.sld_gates_rate, board, self.btn_stop])
        display(screen)
        self.gen_states()
        self.gen_gates()

    def gen_states(self):
        for j in range(SCREEN_Y):
            if j % 2 == 0:
                rangex = range(0, SCREEN_X, 2)
            else:
                rangex = range(1, SCREEN_X, 2)
            for i in rangex:
                self.tiles[j][i].description = random.choice(self.states)
                self.tiles[j][i].style.button_color = self.get_color(self.tiles[j][i].description)

    def gen_gates(self):
        for j in range(SCREEN_Y):
            if j % 2 == 0:
                rangex = range(1, SCREEN_X, 2)
            else:
                rangex = range(0, SCREEN_X, 2)
            for i in rangex:
                self.tiles[j][i].description = random.choice(self.gates)

    def get_gate_up(self, x, y):
        gate_x = x
        gate_y = y - 1
        if gate_x < 0 or gate_x >= SCREEN_X or gate_y < 0 or gate_y >= SCREEN_Y:
            return None
        else:
            return self.tiles[gate_y][gate_x].description
        
    def get_gate_right(self, x, y):
        gate_x = x + 1
        gate_y = y
        if gate_x < 0 or gate_x >= SCREEN_X or gate_y < 0 or gate_y >= SCREEN_Y:
            return None
        else:
            return self.tiles[gate_y][gate_x].description

    def get_gate_down(self, x, y):
        gate_x = x
        gate_y = y + 1
        if gate_x < 0 or gate_x >= SCREEN_X or gate_y < 0 or gate_y >= SCREEN_Y:
            return None
        else:
            return self.tiles[gate_y][gate_x].description

    def get_gate_left(self, x, y):
        gate_x = x - 1
        gate_y = y
        if gate_x < 0 or gate_x >= SCREEN_X or gate_y < 0 or gate_y >= SCREEN_Y:
            return None
        else:
            return self.tiles[gate_y][gate_x].description

    def get_color(self, state):
        if state == '0':
            return 'gray'
        elif state == '1':
            return 'red'
        elif state == '+':
            return 'blue'
        elif state == '-':
            return 'green'
        elif state == '>':
            return 'yellow'
        elif state == '<':
            return 'orange'
        
    def update_states(self):
        for j in range(SCREEN_Y):
            if j % 2 == 0:
                rangex = range(0, SCREEN_X, 2)
            else:
                rangex = range(1, SCREEN_X, 2)
            for i in rangex:
                state = self.tiles[j][i].description
                gate = self.get_gate_up(i, j)
                if gate != None:
                    state = apply_gate_on_state(gate, state)
                gate = self.get_gate_right(i, j)
                if gate != None:
                    state = apply_gate_on_state(gate, state)
                gate = self.get_gate_down(i, j)
                if gate != None:
                    state = apply_gate_on_state(gate, state)
                gate = self.get_gate_left(i, j)
                if gate != None:
                    state = apply_gate_on_state(gate, state)
                self.tiles[j][i].description = state
                self.tiles[j][i].style.button_color = self.get_color(state)

    def on_exit(self):
        self.exit = True

    def loop(self):
        count = 0
        while not self.exit:
            self.update_states()
            count += 1
            if count >= self.sld_gates_rate.value:
                self.gen_gates()
                count = 0
            time.sleep(1.0 / self.sld_frame_rate.value)

qubit_board = QubitBoard()

threading.Thread(target=qubit_board.loop).start()
